In [10]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import time

import numpy as np
import jax
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int, PRNGKeyArray
import aeon
import pandas as pd

from features.sig_trp import SigVanillaTensorizedRandProj, SigRBFTensorizedRandProj
from features.sig import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureTransformer, TabularTimeseriesFeatures, RandomGuesser
from features.sig_neural import RandomizedSignature
from utils import print_name, print_shape

from preprocessing.timeseries_augmentation import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from aeon.classification.sklearn import RotationForestClassifier
from sklearn.metrics import root_mean_squared_error

jax.config.update('jax_platform_name', 'cpu') # Used to set the platform (cpu, gpu, etc.)
np.set_printoptions(precision=3, threshold=5) # Print options

from aeon.datasets.tser_datasets import tser_soton
from aeon.datasets import load_regression

from sklearn.linear_model import RidgeCV
from aeon.transformations.collection.convolution_based import Rocket, MultiRocketMultivariate, MiniRocketMultivariate

from run_tser import do_experiments, get_aeon_dataset

In [8]:
print(len(tser_soton))

for dataset_name in list(tser_soton)[10:]:
    print(dataset_name)
    X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)
    print_shape(X_train)
    print_name(y_train)
    print("\n")

GasSensorArrayAcetone
(324, 7500, 1) X_train 

(324,) y_train
[0.  0.  0.3 ... 0.3 0.  0.3] 



SulphurConcentration
(444, 2307, 1) X_train 

(444,) y_train
[ 7.29  8.48 10.6  ... 21.43  5.83 20.1 ] 



BarCrawl6min
(140, 360, 3) X_train 

(140,) y_train
[0.004 0.05  0.066 ... 0.036 0.013 0.183] 



PhosphorusConcentration
(1573, 3578, 1) X_train 

(1573,) y_train
[ 3.238  8.48   6.36  ...  4.593  2.075 17.8  ] 



MagnesiumConcentration
(1560, 3578, 1) X_train 

(1560,) y_train
[103.   100.    43.6  ...  49.76 109.2  110.  ] 



ElectricityPredictor
(567, 168, 4) X_train 

(567,) y_train
[ 58.491 156.668 181.468 ...  96.835  80.132 246.16 ] 



EthereumSentiment
(249, 24, 2) X_train 

(249,) y_train
[-0.042  0.549  0.248 ...  0.659  0.384  0.464] 



BIDMC32HR


KeyboardInterrupt: 

In [ ]:
#run experiments
d_res, d_meta, d_failed = do_experiments(list(tser_soton))

# make dict of results
model_names = d_res[list(d_res.keys())[0]][0][0]
alpha_names = ["alpha_" + model_name for model_name in model_names]
df_RMSEs = pd.DataFrame({dataset : RMSEs for dataset, ((model_names, RMSEs, alphas),) in d_res.items()}).transpose()
df_RMSEs.columns = model_names
df_alphas = pd.DataFrame({dataset : alphas for dataset, ((model_names, RMSEs, alphas),) in d_res.items()}).transpose()
df_alphas.columns = alpha_names
meta = pd.DataFrame(d_meta).transpose()

df_RMSEs = pd.concat([meta, df_RMSEs], axis=1)
df_alphas = pd.concat([meta, df_alphas], axis=1)

# save
df_RMSEs.to_pickle("df_RMSEs_TSER.pkl")
df_alphas.to_pickle("df_alphas_TSER.pkl")
print(d_failed)

In [12]:
df_RMSEs = pd.read_pickle("df_accs_TSER.pkl")
df_RMSEs


,N_train,N_test,T,D,Random Guesser,Tabular,Sig,Log Sig,Sig Vanilla TRP,Sig RBF TRP,Randomized Signature,Rocket,MiniRocket,MultiRocket
PrecipitationAndalusia,470,202,365,4,1.279958,0.849867,1.748869,1.034546,1.259405,0.958332,2.524139,0.974973,0.891474,0.962430
BarCrawl6min,140,61,360,3,0.980592,13.144074,4.680805,3.874491,11.078803,7.668395,2.325620,4.010398,2.010791,1.995112
ElectricityPredictor,567,243,168,4,1.202188,1.946083,2.130031,1.308071,1.422916,1.978265,3.733057,1.522119,1.513569,1.470004
